In [ ]:
import streamlit as st
from langchain_community.llms import Ollama
from langchain_community.embeddings import OllamaEmbeddings
from langchain_community.vectorstores import FAISS
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnablePassthrough

# --- Configuration and Setup ---

# Initialize Ollama for LLM and embeddings (ensure Ollama server is running and models are pulled)
# You can replace 'llama2' and 'nomic-embed-text' with other models you have.
try:
    llm = Ollama(model="llama2")
    embeddings = OllamaEmbeddings(model="nomic-embed-text")
except Exception as e:
    st.error(f"Error initializing Ollama. Make sure Ollama is running and models are pulled: {e}")
    st.stop() # Stop the app if Ollama cannot be initialized

# Define some dummy documents for demonstration
# In a real RAG app, these would come from a data source (e.g., loaded from files, databases)
docs = [
    "The capital of France is Paris. Paris is known for its beautiful architecture.",
    "Mount Everest is the highest mountain in the world, located in the Himalayas.",
    "The Eiffel Tower is a famous landmark in Paris, France.",
    "Deep learning is a subset of machine learning inspired by the structure of the human brain.",
    "Python is a popular programming language for data science and AI."
]

# Create a FAISS vector store from the dummy documents
# In a production environment, you would load an existing vector store
try:
    vectorstore = FAISS.from_texts(docs, embeddings)
    retriever = vectorstore.as_retriever()
except Exception as e:
    st.error(f"Error creating vector store: {e}")
    st.stop()

# --- LangChain Prompt and Chain Definition ---

# Define the RAG prompt template
# We use MessagesPlaceholder to dynamically inject chat history
prompt = ChatPromptTemplate.from_messages(
    [
        ("system", "You are a helpful AI assistant. Answer the user's questions based on the provided context. If you don't know the answer, say 'I don't have enough information to answer that.'"),
        MessagesPlaceholder(variable_name="chat_history"), # Placeholder for chat history
        ("user", "Context: {context}\n\nQuestion: {question}"),
    ]
)

# Define the RAG chain
# We define a function to format documents for the context
def format_docs(docs):
    return "\n\n".join(doc.page_content for doc in docs)

rag_chain = (
    RunnablePassthrough.assign(context=(lambda x: x["question"]) | retriever | format_docs) # Retrieve context based on question
    | prompt # Apply the prompt template
    | llm # Pass to the LLM
    | StrOutputParser() # Parse the output
)

# --- Streamlit Application ---

st.set_page_config(page_title="RAG LLM Chat with History", layout="centered")
st.title("RAG LLM Chat with History")
st.info("This RAG app uses Ollama for the LLM and embeddings, and stores chat history in Streamlit's session state. Ensure your Ollama server is running and models ('llama2', 'nomic-embed-text') are pulled.")

# Initialize chat history in Streamlit's session state if it doesn't exist
if "messages" not in st.session_state:
    st.session_state.messages = []

# Display chat messages from history on app rerun
for message in st.session_state.messages:
    with st.chat_message(message["role"]):
        st.markdown(message["content"])

# Accept user input
if prompt_input := st.chat_input("Ask a question about the documents..."):
    # Add user message to chat history
    st.session_state.messages.append({"role": "user", "content": prompt_input})
    with st.chat_message("user"):
        st.markdown(prompt_input)

    # Prepare chat history for the RAG chain
    # We need to convert Streamlit's message format to LangChain's message format
    langchain_chat_history = []
    for msg in st.session_state.messages:
        if msg["role"] == "user":
            langchain_chat_history.append({"role": "user", "content": msg["content"]})
        elif msg["role"] == "assistant":
            langchain_chat_history.append({"role": "assistant", "content": msg["content"]})

    # Call the RAG chain with the current question and chat history
    with st.chat_message("assistant"):
        with st.spinner("Thinking..."):
            # The rag_chain expects 'question' and 'chat_history' as input
            response = rag_chain.invoke({
                "question": prompt_input,
                "chat_history": langchain_chat_history # Pass the entire chat history
            })
        st.markdown(response)

    # Add assistant response to chat history
    st.session_state.messages.append({"role": "assistant", "content": response})

